# Primefactor

In [5]:
from __future__ import print_function
import numpy as np

#The prime factors of 13195 are 5, 7, 13 and 29.

#What is the largest prime factor of the number 600851475143 ?


N = 600851475143
LIM = 10 ** 6

def factor(n):
   #1. Create array of trial values
   a = np.ceil(np.sqrt(n))
   lim = min(n, LIM)
   a = np.arange(a, a + lim)
   b2 = a ** 2 - n

   #2. Check whether b is a square
   fractions = np.modf(np.sqrt(b2))[0]
   
   """ np.modf() function separates fractional and integer part of a number and returns [fraction, integer] """

   #3. Find 0 fractions
   indices = np.where(fractions == 0)

   #4. Find the first occurrence of a 0 fraction
   a = np.ravel(np.take(a, indices))[0]

   """ np.ravel returns one-dimension array and np.take takes element in given index """

   # Or a = a[indices][0]

   a = int(a)
   b = np.sqrt(a ** 2 - n) 
   b = int(b)
   c = a + b
   d = a - b

   if c == 1 or d == 1:
      return

   print(c, d)
   factor(c)
   factor(d)

factor(N)
#1234169 486847
#1471 839
#6857 71

1234169 486847
1471 839
6857 71


# Palindrome

In [6]:
import numpy as np

#A palindromic number reads the same both ways. 
#The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 x 99.

#Find the largest palindrome made from the product of two 3-digit numbers.


#1. Create  3-digits numbers array
a = np.arange(100, 1000)
np.testing.assert_equal(100, a[0])
np.testing.assert_equal(999, a[-1])

#2. Create products array
numbers = np.outer(a, a)
numbers = np.ravel(numbers)
numbers.sort()
np.testing.assert_equal(810000, len(numbers))
np.testing.assert_equal(10000, numbers[0])
np.testing.assert_equal(998001, numbers[-1])

#3. Find largest palindromic number
for number in numbers[::-1]:
   s = str(number)

   """ np.array[::-1] reverses array """
   
   if s == s[::-1]:
      print(s)
      break

906609


# Steady state vector

In [7]:
from __future__ import print_function
from matplotlib.finance import quotes_historical_yahoo_ohlc
from datetime import date
import numpy as np


today = date.today()
start = (today.year - 1, today.month, today.day)

# Get close price (Yahoo's stock price)
quotes = quotes_historical_yahoo_ohlc('AAPL', start, today)
close =  [q[4] for q in quotes]

# Get discrete differences (cost[date + 1] - cost[date])
states = np.sign(np.diff(close))

# Initialize Stochastic matrix for Ax = x in Markov chain
NDIM = 3
SM = np.zeros((NDIM, NDIM))

signs = [-1, 0, 1] # [down, flat, up]
k = 1

for i, signi in enumerate(signs):
   #we start the transition from the state with the specified sign
   start_indices = np.where(states[:-1] == signi)[0]
   print(signi)
   print(start_indices)
   N = len(start_indices) + k * NDIM

   # skip since there are no transitions possible
   if N == 0:
      continue
   
   #find the values of states at the end positions
   end_values = states[start_indices + 1] # Get results of each transition
   print("dfdf",end_values)
   for j, signj in enumerate(signs):
      # number of occurrences of this transition 
      occurrences = len(end_values[end_values == signj])
      SM[i][j] = (occurrences + k)/float(N)

print(SM)
"""
    SM = [[P(-1 to -1), P(-1 to 0), P(-1 to 1)],[P(0 to -1), P(0 to 0), P(0 to 1)],[P(1 to -1), P(1 to 0), P(1 to 1)]
"""
eig_out = np.linalg.eig(SM) # Get eigenvector
print(eig_out)

idx_vec = np.where(np.abs(eig_out[0] - 1) < 0.1)
print("Index eigenvalue 1", idx_vec)

x = eig_out[1][:,idx_vec].flatten()
# Check x for SM*x = x
print("Steady state vector", x)
print("Check", np.dot(SM, x))


HTTPError: HTTP Error 404: Not Found

# Power law

In [8]:
from matplotlib.finance import quotes_historical_yahoo_ohlc
from datetime import date
import numpy as np
import matplotlib.pyplot as plt

#1. Get close prices.
today = date.today()
start = (today.year - 1, today.month, today.day)

quotes = quotes_historical_yahoo_ohlc('IBM', start, today)
close =  np.array([q[4] for q in quotes])

#2. Get positive log returns.
logreturns = np.diff(np.log(close))
pos = logreturns[logreturns > 0]

#3. Get frequencies of returns.
counts, rets = np.histogram(pos)
print(counts)
print(rets)
# 0 counts indices
indices0 = np.where(counts != 0)
rets = rets[:-1] + (rets[1] - rets[0])/2
# Could generate divide by 0 warning
freqs = 1.0/counts
freqs = np.take(freqs, indices0)[0]
rets = np.take(rets, indices0)[0]
freqs =  np.log(freqs)

#4. Fit the frequencies and returns to a line.
p = np.polyfit(rets,freqs, 1)

#5. Plot the results.
plt.title('Power Law')
plt.plot(rets, freqs, 'o', label='Data')
plt.plot(rets, p[0] * rets + p[1], label='Fit')
plt.xlabel('Log Returns')
plt.ylabel('Log Frequencies')
plt.legend() # explanation for each plot
plt.grid()
plt.show()

HTTPError: HTTP Error 404: Not Found

# Periodic

In [9]:
from __future__ import print_function
from matplotlib.finance import quotes_historical_yahoo_ohlc
from datetime import date
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

#1. Get close prices.
today = date.today()
start = (today.year - 1, today.month, today.day)

quotes = quotes_historical_yahoo_ohlc('AAPL', start, today)
close =  np.array([q[4] for q in quotes])

#2. Get log returns.
logreturns = np.diff(np.log(close))

#3. Calculate breakout and pullback
freq = 0.02
breakout = scipy.stats.scoreatpercentile(logreturns, 100 * (1 - freq) ) # scipy.stats.scoreatpercentile gets score at given percentile
pullback = scipy.stats.scoreatpercentile(logreturns, 100 * freq)

#4. Generate buys and sells
buys = np.compress(logreturns < pullback, close) # return matrix where logreturns < pullback
sells = np.compress(logreturns > breakout, close) # this equals to close[logreturns > breakout]
print(buys)
print(sells)
print(len(buys), len(sells))
print(sells.sum() - buys.sum())

#5. Plot a histogram of the log returns
plt.title('Periodic Trading')
plt.hist(logreturns)
plt.grid()
plt.xlabel('Log Returns')
plt.ylabel('Counts')
plt.show()

HTTPError: HTTP Error 404: Not Found

# Random periodic

In [10]:
from __future__ import print_function
from matplotlib.finance import quotes_historical_yahoo
from datetime import date
import numpy as np
import matplotlib.pyplot as plt

def get_indices(high, size):
   #2. Generate random indices
   return np.random.randint(0, high, size)

#1. Get close prices.
today = date.today()
start = (today.year - 1, today.month, today.day)

quotes = quotes_historical_yahoo('AAPL', start, today)
close =  np.array([q[4] for q in quotes])

nbuys = 5
N = 2000
profits = np.zeros(N)

for i in xrange(N):
   #3. Simulate trades
   buys = np.take(close, get_indices(len(close), nbuys))
   sells = np.take(close, get_indices(len(close), nbuys))
   profits[i] = sells.sum() - buys.sum()

print("Mean", profits.mean())
print("Std", profits.std())

#4. Plot a histogram of the profits
plt.title('Simulation')
plt.hist(profits) # Plot histogram on array
plt.xlabel('Profits')
plt.ylabel('Counts')
plt.grid()
plt.show()

ImportError: cannot import name 'quotes_historical_yahoo'

# Erastothenes

In [11]:
from __future__ import print_function
import numpy as np

LIM = 10 ** 6
N = 10 ** 9
P = 10001
primes = []
p = 2

#By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13.

#What is the 10 001st prime number?

def sieve_primes(a, p):
   #2. Sieve out multiples of p
   a = a[a % p != 0]

   return a

for i in range(3, N, LIM):
   #1. Create a list of consecutive integers
   a = np.arange(i, i + LIM, 2) # They put 2 to sieve out even numbers in the first place
   print(a)
   # get prime numbers in interval (e.g. 3 ~ 1000001, 1000003 ~ 2000001)
   while len(primes) < P:
      a = sieve_primes(a, p)
      primes.append(p)

      p = a[0]

print(len(primes), primes[P-1])

[      3       5       7 ...,  999997  999999 1000001]
[1000003 1000005 1000007 ..., 1999997 1999999 2000001]
[2000003 2000005 2000007 ..., 2999997 2999999 3000001]
[3000003 3000005 3000007 ..., 3999997 3999999 4000001]
[4000003 4000005 4000007 ..., 4999997 4999999 5000001]
[5000003 5000005 5000007 ..., 5999997 5999999 6000001]
[6000003 6000005 6000007 ..., 6999997 6999999 7000001]
[7000003 7000005 7000007 ..., 7999997 7999999 8000001]
[8000003 8000005 8000007 ..., 8999997 8999999 9000001]
[ 9000003  9000005  9000007 ...,  9999997  9999999 10000001]
[10000003 10000005 10000007 ..., 10999997 10999999 11000001]
[11000003 11000005 11000007 ..., 11999997 11999999 12000001]
[12000003 12000005 12000007 ..., 12999997 12999999 13000001]
[13000003 13000005 13000007 ..., 13999997 13999999 14000001]
[14000003 14000005 14000007 ..., 14999997 14999999 15000001]
[15000003 15000005 15000007 ..., 15999997 15999999 16000001]
[16000003 16000005 16000007 ..., 16999997 16999999 17000001]
[17000003 1700000